In [1]:
using Pkg
pkg"activate ."

 Activating environment at `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Project.toml`


In [2]:
pkg"add PyCall" # to run Python code from Julia
pkg"add BenchmarkTools" # to run benchmarks

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Project.toml`
 [no changes]
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Project.toml`
 [no changes]
   Updating `/mnt/SSD_Data/active_projects/transformer_to_lstm/notebooks/Manifest.toml`
 [no changes]


In [3]:
using PyCall
py"""
import nlp
import numpy as np
from transformers import BertTokenizer

def extract_sentences():
    dataset = nlp.load_dataset('glue', "sst2")
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    results = []
    for sent in dataset['train']['sentence']:
        results.append(tokenizer.tokenize(sent))
    return results
"""

In [4]:
@time sentences = py"extract_sentences"();

 15.095235 seconds (3.77 M allocations: 140.210 MiB, 1.29% gc time)


In [5]:
sentences = vcat(sentences, sentences, sentences, sentences) # x4 sizes

269396-element Array{Array{String,1},1}:
 ["hide", "new", "secret", "##ions", "from", "the", "parental", "units"]
 ["contains", "no", "wit", ",", "only", "labor", "##ed", "gag", "##s"]
 ["that", "loves", "its", "characters", "and", "communicate", "##s", "something", "rather", "beautiful", "about", "human", "nature"]
 ["remains", "utterly", "satisfied", "to", "remain", "the", "same", "throughout"]
 ["on", "the", "worst", "revenge", "-", "of", "-", "the", "-", "ne", "##rds", "cl", "##iche", "##s", "the", "filmmakers", "could", "dr", "##edge", "up"]
 ["that", "'", "s", "far", "too", "tragic", "to", "merit", "such", "superficial", "treatment"]
 ["demonstrates", "that", "the", "director", "of", "such", "hollywood", "blockbuster", "##s", "as"  …  "small", ",", "personal", "film", "with", "an", "emotional", "wall", "##op", "."]
 ["of", "sa", "##uc", "##y"]
 ["a", "depressed", "fifteen", "-", "year", "-", "old", "'", "s", "suicidal", "poetry"]
 ["are", "more", "deeply", "thought", "through", "

In [6]:
function add_special_tokens!(sentence)
    pushfirst!(sentence, "[CLS]")
    push!(sentence, "[SEP]")
end

add_special_tokens! (generic function with 1 method)

In [7]:
using BenchmarkTools
@benchmark results = add_special_tokens!.(tmp) setup=(tmp=deepcopy(sentences))

BenchmarkTools.Trial: 
  memory estimate:  20.33 MiB
  allocs estimate:  105978
  --------------
  minimum time:     9.757 ms (0.00% GC)
  median time:      16.849 ms (0.00% GC)
  mean time:        19.532 ms (38.26% GC)
  maximum time:     109.767 ms (85.54% GC)
  --------------
  samples:          32
  evals/sample:     1

In [8]:
function is_first_piece(arr::Array{String,1})
    return .!startswith.(arr, "##")
end

is_first_piece (generic function with 1 method)

In [9]:
@benchmark results = is_first_piece.(tmp) setup=(tmp=deepcopy(sentences)) # doesn't really need to deepcopy, but just to be safe

BenchmarkTools.Trial: 
  memory estimate:  34.94 MiB
  allocs estimate:  538796
  --------------
  minimum time:     32.263 ms (0.00% GC)
  median time:      35.878 ms (0.00% GC)
  mean time:        45.798 ms (21.29% GC)
  maximum time:     134.499 ms (69.26% GC)
  --------------
  samples:          31
  evals/sample:     1

In [10]:
results = is_first_piece.(sentences)

269396-element Array{BitArray{1},1}:
 [1, 1, 1, 0, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 0, 1, 0]
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
 [1, 1, 0, 0]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1]
 [1, 1, 1, 0, 0, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1]
 ⋮
 [1, 1, 1, 0, 1, 1, 0, 0]
 [1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 0]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1]
 [1, 1, 1]
 [1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1]
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
Threads.nthreads()

4

In [12]:
@benchmark begin
    results = [Bool[] for _ in 1:length(sentences)]
    Threads.@threads for i in 1:length(sentences)
       results[i] = is_first_piece(sentences[i])
    end
end

BenchmarkTools.Trial: 
  memory estimate:  89.64 MiB
  allocs estimate:  1346499
  --------------
  minimum time:     20.399 ms (0.00% GC)
  median time:      43.885 ms (43.87% GC)
  mean time:        37.039 ms (35.70% GC)
  maximum time:     64.366 ms (44.26% GC)
  --------------
  samples:          135
  evals/sample:     1

In [13]:
# results = [BitArray(undef, length(sentences[i])) for i in 1:length(sentences)]
results = [Bool[] for _ in 1:length(sentences)]
Threads.@threads for i in 1:length(sentences)
   results[i] = is_first_piece(sentences[i])
end
results[1]

8-element Array{Bool,1}:
 1
 1
 1
 0
 1
 1
 1
 1